In [183]:
from __future__ import division
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [184]:
import os
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize']=8,5

In [185]:
#import data
os.chdir("/home/gladkov/storage/al_R/3new")
os.listdir()
otus = pd.read_csv("otu_table_for_rf_merged.txt", sep="\t")
metadata = pd.read_csv("metadata_for_rf_merged.txt", sep="\t")



In [125]:
# some function for more suitable rownames
def rownames_creator(x):
    first_column = x[0]
    repeats_column = x[2]
    repeat_number = first_column.split(".")[1]
    rownames = repeats_column + "." + repeat_number
    return rownames
# return first_column
metadata["rownames"] = metadata.apply(rownames_creator, axis=1, result_type="expand")
metadata = metadata.rename({metadata.columns[0]: "rowname_old"}, axis="columns")
metadata = metadata.set_index("rownames")
meta_col = list(metadata.columns)

# repair otu table & create nicy rownames like above
# otus.columns = otus.iloc[0]
# otus = otus.drop(["Unnamed: 0"])
# not any need to repair?
# otus.index = meta_col
meta_index = list(metadata.index)
otus.index = meta_index
otus = otus.drop(columns=["Unnamed: 0"])

In [186]:
# same shit that above with merged repeats
metadata.index = metadata["Unnamed: 0"]
metadata = metadata.drop(columns=["Unnamed: 0"])

otus.index = otus["Unnamed: 0"]
otus = otus.drop(columns=["Unnamed: 0"])

metadata = metadata.drop(columns=["Filename","Description","Plant","Inoculation","Al", "QRTPCR"])

In [181]:
#create df with factor that consist only numerical data
met_col_list = list(metadata.columns)
count_list = list(range(0,105))
# np.column_stack((count_list, met_col_list))
# metadata = metadata.drop([])


In [187]:
metadata

,BineM,SeedM,BineSeedM,SeedN,SeedMOne,NodPerPlant,NodPerPlant.1,Mic,Ves,Arb,...,BineSeedPerPlantCu,BineSeedPerPlantFe,BineSeedPerPlantK,BineSeedPerPlantMg,BineSeedPerPlantMo,BineSeedPerPlantNi,BineSeedPerPlantP,BineSeedPerPlantS,BineSeedPerPlantS.1,BineSeedPerPlantZn
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
s1903,5.217500,4.342500,9.1975,40.5,121.156977,37.5,11.30,52.0000,19.35,32.00,...,205.559426,766.378262,714.277514,62.535185,76.387623,147.119788,80.156457,189.861096,47.650990,162.618635
s1903_Al,4.392500,3.815000,8.3575,32.5,116.472538,8.0,3.00,3.5375,3.50,2.65,...,180.729942,666.361386,693.159352,55.043641,73.033851,296.895381,20.194510,163.559044,40.544464,152.699049
s1903_In,5.927500,5.102500,11.0300,38.5,120.505128,75.0,22.00,41.0000,19.35,17.65,...,241.547593,1020.152172,756.427953,81.277633,90.928284,186.786847,113.749773,218.738423,59.010729,317.804489
s1903_In_Al,6.280000,4.510000,10.7600,38.0,132.236842,107.0,25.00,2.3000,6.00,4.00,...,214.902651,882.726277,1024.193155,85.113614,102.978868,432.093049,31.304417,204.789880,69.358599,197.320157
s7307,6.171667,9.320833,15.7325,37.0,245.901568,76.0,40.55,27.6500,15.00,10.00,...,349.903165,1228.812499,610.736871,79.297033,118.448587,197.922382,54.625197,346.483515,50.242766,314.062153
s7307_Al,5.790000,6.465000,12.2875,29.5,222.635417,89.0,25.90,44.6500,13.30,36.35,...,328.323829,1348.291963,814.213264,82.019465,193.789526,511.311272,30.422035,316.907947,56.710160,514.203436
s7307_In,7.550000,9.617500,17.2850,36.0,267.682228,109.0,50.00,37.1500,15.65,21.30,...,425.149237,1716.719807,964.578965,93.713259,165.309027,283.260445,84.401096,422.719562,70.984900,450.168242
s7307_In_Al,6.807500,6.432500,13.3475,28.5,229.732143,211.0,37.85,64.1500,42.65,41.30,...,309.794086,1067.851311,831.875468,92.289192,179.595986,565.965374,30.928963,298.532203,62.610131,426.562850
s8353,3.290000,4.607500,7.8975,19.5,248.727941,26.5,10.90,50.8500,17.85,44.00,...,217.580950,642.869897,450.987282,45.250730,112.997601,219.508415,96.767008,264.529665,50.103618,203.462065


In [188]:
rf = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
# rf(otus, metadata)
X_train, X_valid, y_train, y_valid = train_test_split(otus, metadata, test_size = 0.8, random_state = 42)
rf.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [133]:
?otus.iloc[]

Object `otus.iloc[]` not found.
